# 🤖 **Training GPT for Instruction Following**

In this notebook, we demonstrate the process of training a GPT model to follow instructions effectively, using a custom dataset for varied tasks. Let's embark on this journey from data preprocessing to model training and text generation.


## Setup and Dependencies

First, we import necessary libraries and set up our environment to handle the tasks.

In [ ]:
!pip install transformers[torch]==4.38.2
!pip install datasets===2.13.1

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset



## Data Exploration and Preparation 📊

Let's load and preview our dataset, ensuring we understand the kind of data we're working with.


## Data Exploration and Preparation 📊

Let's load and preview our dataset, ensuring we understand the kind of data we're working with.

In [ ]:

dataset = load_dataset("hakurei/open-instruct-v1", split='train')
dataset.to_pandas().sample(20)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,input,instruction,output
43951,,Identify a major international problem that pe...,One major international problem that people ar...
291954,,Identify a key feature of the web development ...,One key feature of the web development process...
174107,,What is the capital of Costa Rica.,The capital of Costa Rica is San Jose.
158868,,How do I plant a row of wildflowers.,Choose an appropriate location for the wildflo...
14578,,Write a fable using animals.,"Once upon a time, there was a wise old lion wh..."
267062,"In today's digital world, it is important for ...",Take the content provided and rewrite it in 45...,A Company's success is determined by its custo...
201811,,Write a regex (in Python) to match the followi...,"import re\n\nregex = r""\ban\sapple\s\ba\sday\s..."
318040,The sun is shining and the birds are singing.,Translate the following sentence into Spanish.,El sol brilla y los pájaros cantan.
332806,,Find out how many people are on your facebook ...,def countFriendsInSameCity(self):\n friends...
391796,,"Tell me if this sentence is true or false: ""al...",False


In [ ]:

def preprocess(example):
    example['prompt'] = f"{example['instruction']} {example['input']} {example['output']}"

    return example

def tokenize_datasets(dataset):
    tokenized_dataset = dataset.map(lambda example: tokenizer(example['prompt'], truncation=True, max_length=128), batched=True, remove_columns=['prompt'])

    return tokenized_dataset




### Shuffling and Splitting the Dataset

Next, we shuffle the dataset and split it into training and test sets to ensure robust model training and evaluation.

In [ ]:
dataset = dataset.map(preprocess, remove_columns=['instruction', 'input', 'output'])
dataset =  dataset.shuffle(42).select(range(100000)).train_test_split(test_size=0.1, seed=42)

In [ ]:

train_dataset = dataset['train']
test_dataset = dataset['test']

## Model Initialization and Tokenization 🚀

We set up the tokenizer and the model, ensuring that our tokens align with the model's expected format.

In [ ]:

MODEL_NAME = "microsoft/DialoGPT-medium"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

train_dataset = tokenize_datasets(train_dataset)
test_dataset = tokenize_datasets(test_dataset)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)


## Training the GPT Model 🎯
Now, we configure the training parameters and initiate the training process using our prepared datasets.

In [ ]:

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

traing_args = TrainingArguments(output_dir="models/diablo_gpt",
                                num_train_epochs=1,
                                per_device_train_batch_size=32,
                                per_device_eval_batch_size=32)\

trainer = Trainer(model=model,
                    args=traing_args,
                    train_dataset=train_dataset,
                    eval_dataset=test_dataset,
                    data_collator=data_collator)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train() # this will take a long time


In [ ]:
# Get the trained checkpoint directly
model = AutoModelForCausalLM.from_pretrained("TheFuzzyScientist/diabloGPT_open-instruct")

config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

## Text Generation and Application 📝
Finally, let's demonstrate the capability of our trained model by generating responses to various instructions.

In [ ]:

def generate_text(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt') #.to("cuda") # <-- if running on GPU, uncomment this
    outputs = model.generate(inputs, max_length=64, pad_token_id=tokenizer.eos_token_id)
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated[:generated.rfind('.')+1]

In [ ]:
generate_text("What's the best way to cook chiken breast?")

"What's the best way to cook chiken breast?  The best way to cook chiken breast is to season it with salt and pepper, then heat a pan over medium heat. Add a tablespoon of olive oil and cook for about 5 minutes, stirring occasionally."

In [ ]:
generate_text("Should I invest stocks?")


'Should I invest stocks?  Yes, it is a good idea to invest in stocks. It is important to understand the risks associated with investing in stocks and to make sure that you are taking the necessary precautions. It is also important to understand the potential returns and to make sure that you are making the right investment.'

In [ ]:
generate_text("I need a place to go for this summer vacation, what locations would you recommend")


'I need a place to go for this summer vacation, what locations would you recommend.  I would recommend visiting the beach in San Diego, California. It is a popular destination for vacationers and has a great view of the ocean.'

In [ ]:
generate_text("What's the fastest route from NY City to Boston?")


"What's the fastest route from NY City to Boston?  The fastest route from New York City to Boston is by taking the New York City subway. The subway takes about 3 hours and 15 minutes to get from the city center to the Boston Common."


## Conclusion and Further Exploration 🌟

This notebook guides you through training a GPT model to follow complex instructions. Experiment further by altering the dataset or tweaking the model and training configurations to explore new possibilities!